In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import plotly.express as px
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nifty50actual/NIFTY50.csv


In [17]:
data = pd.read_csv("/kaggle/input/nifty50actual/NIFTY50.csv")
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-05-04,9533.500000,9533.500000,9266.950195,9293.500000,9293.500000,687500.0
1,2020-05-05,9429.400391,9450.900391,9190.750000,9205.599609,9205.599609,725200.0
2,2020-05-06,9226.799805,9346.900391,9116.500000,9270.900391,9270.900391,722200.0
3,2020-05-07,9234.049805,9277.849609,9175.900391,9199.049805,9199.049805,708700.0
4,2020-05-08,9376.950195,9382.650391,9238.200195,9251.500000,9251.500000,609100.0


In [3]:
data.set_index('Date', inplace = True)

In [4]:
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)

In [19]:
print(data.isna().sum())
data.dropna(inplace = True)

Date         0
Open         2
High         2
Low          2
Close        2
Adj Close    2
Volume       2
dtype: int64


In [72]:
fig = px.line(data, x="Date", y="Open", title='Interactive NIFTY50 Close Price')
fig.show()

In [55]:
#Scaling the features values in the range of 0 to 1

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0,1))

In [102]:
training_data =data[['Open']]

In [118]:
feature_scaled = scaler.fit_transform(training_data)

In [146]:
#Using 60 days to predict 61'st day
X = []
y = []
for i in range(60, len(training_data)):
    X.append(feature_scaled[i-60:i,0])
    y.append(feature_scaled[i,0])

# Converting list into a numpy array     
X,y = np.array(X), np.array(y)
    
    

In [147]:
len(X)

189

In [148]:
X_train = X[:-7]
X_test = X[-7:]
y_train = y[:-7]
y_test = y[-7:]

In [149]:
X_train.shape

(182, 60)

In [150]:
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [151]:
#Building LSTM Sequential model
import tensorflow as tf

In [152]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [153]:
model = Sequential()
model.add(LSTM(units = 150,return_sequences = True, input_shape = (X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units = 100,return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 100,return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 100,return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 150,return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
model.add(Dense(units = 20))
model.add(Dense(units = 1))
model.compile(optimizer= 'adam',loss = 'mean_squared_error')

In [154]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 60, 150)           91200     
_________________________________________________________________
dropout_24 (Dropout)         (None, 60, 150)           0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 60, 100)           100400    
_________________________________________________________________
dropout_25 (Dropout)         (None, 60, 100)           0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 60, 100)           80400     
_________________________________________________________________
dropout_26 (Dropout)         (None, 60, 100)           0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 60, 100)          

In [155]:
model.fit(X_train,y_train,epochs=100,batch_size = 32)

Epoch 1/100
6/6 [==============================] - 11s 356ms/step - loss: 0.3504
Epoch 2/100
6/6 [==============================] - 2s 368ms/step - loss: 0.1860
Epoch 3/100
6/6 [==============================] - 2s 386ms/step - loss: 0.0946
Epoch 4/100
6/6 [==============================] - 2s 352ms/step - loss: 0.0451
Epoch 5/100
6/6 [==============================] - 2s 366ms/step - loss: 0.0359
Epoch 6/100
6/6 [==============================] - 2s 350ms/step - loss: 0.0273
Epoch 7/100
6/6 [==============================] - 2s 351ms/step - loss: 0.0203
Epoch 8/100
6/6 [==============================] - 2s 351ms/step - loss: 0.0187
Epoch 9/100
6/6 [==============================] - 2s 356ms/step - loss: 0.0138
Epoch 10/100
6/6 [==============================] - 2s 361ms/step - loss: 0.0113
Epoch 11/100
6/6 [==============================] - 2s 350ms/step - loss: 0.0080
Epoch 12/100
6/6 [==============================] - 2s 360ms/step - loss: 0.0103
Epoch 13/100
6/6 [==================

In [156]:
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test.reshape(y_pred.shape[0],1))

In [157]:
y_pred = y_pred.reshape(y_pred.shape[0],)
y_test = y_test.reshape(y_test.shape[0],)


In [158]:
df =pd.DataFrame({"actual_stock_open":y_test,"predicted_stock_open":y_pred})
df

,actual_stock_open,predicted_stock_open
0,14326.349609,14506.887695
1,14449.450195,14486.201172
2,14493.799805,14470.202148
3,14710.500000,14458.182617
4,14979.000000,14451.465820
5,14747.349609,14452.070312
6,14481.049805,14453.213867
